In [ ]:
import os
import autogen
import json
from dotenv import load_dotenv
load_dotenv()


import datetime
import os

# custom_autogen_model package
from custom_autogen_model import CustomModelClient


# pyautogen package
from autogen import ConversableAgent, UserProxyAgent, AssistantAgent
from autogen.coding import LocalCommandLineCodeExecutor


: 

In [2]:
def config_list(config, client_name="CustomModelClient"):
    dict_list = [config]

    # set the config into an environment variable
    os.environ["IMAGINE_AUTOGEN_CONFIG"] = json.dumps(dict_list)

    config_list_custom = autogen.config_list_from_json(
        "IMAGINE_AUTOGEN_CONFIG",
        filter_dict={"model_client_cls": [client_name]},
    )
    return config_list_custom

In [3]:
api_key = os.environ["IMAGINE_API_KEY"]

config = {
    "model": "imaginechat",
    "model_client_cls": "CustomModelClient",
    "n": 1,
    "params": {"max_length": 1500, "api_key": api_key, "temperature": 0.0},
}

config_list_custom = config_list(config)

In [4]:
executor = LocalCommandLineCodeExecutor(
    timeout=60,
    work_dir="coding",
)

In [5]:
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    code_execution_config={"executor": executor},
    default_auto_reply="Please continue. If everything is done, reply 'TERMINATE'.",
)

In [6]:
code_writer_agent = UserProxyAgent(
    name="code_writer_agent",
    llm_config={"config_list": config_list_custom},
    code_execution_config=False,
)

# code_writer_agent = AssistantAgent(
#     name="code_writer_agent",
#     llm_config={"config_list": config_list_custom},
#     code_execution_config=False,
#     human_input_mode="NEVER",
# )

[autogen.oai.client: 11-18 19:23:37] {565} INFO - Detected custom model client in config: CustomModelClient, model client can not be used until register_model_client is called.


In [7]:
# register the custom client class to the agents
code_writer_agent.register_model_client(model_client_cls=CustomModelClient)

CustomModelClient config: {'model': 'imaginechat', 'model_client_cls': 'CustomModelClient', 'n': 1, 'params': {'max_length': 1500, 'api_key': '38e48000-4d41-4a15-9cfb-e97056b06eb8', 'temperature': 0.0}}
Loaded model imaginechat


In [8]:
code_writer_agent_system_message = code_writer_agent.system_message
print(code_writer_agent_system_message)

In [9]:
# Ask the two agents to collaborate on a stock analysis task.
today = datetime.datetime.now().date()
message = (
    f"Today is {today}. "
    "Create a plot showing stock gain YTD over the whole year for (Qualcomm) QCOM and (Tesla) TSLA. "
    "Make sure the code is in python code block and save the figure"
    " to a file ytd_stock_gains.png."
    "use the yfinance lib"
    ""
)

In [10]:
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message=message,
)

code_executor_agent (to code_writer_agent):

Today is 2024-11-18. Create a plot showing stock gain YTD over the whole year for (Qualcomm) QCOM and (Tesla) TSLA. Make sure the code is in python code block and save the figure to a file ytd_stock_gains.png.use the yfinance lib

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
response:  namespace(choices=[namespace(message=namespace(content=AIMessage(content="Here is a Python code block that uses the yfinance library to retrieve the stock data for Qualcomm (QCOM) and Tesla (TSLA) and plots the year-to-date (YTD) stock gains:\n```python\nimport yfinance as yf\nimport matplotlib.pyplot as plt\n\n# Define the stock tickers\ntickers = ['QCOM', 'TSLA']\n\n# Get the historical data for each stock\nfor ticker in tickers:\n    stock_data = yf.download(ticker, start='2022-01-01', end='2024-11-18')\n\n    # Calculate the YTD gain\n    ytd_gain = (stock_